In [1]:
import os

In [2]:
%pwd

'c:\\Users\\ADMIN\\Desktop\\Euron\\MLOps\\7_Advanced_ML_Pipeline\\End-to-End-Machine-Learning-Pipeline\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\ADMIN\\Desktop\\Euron\\MLOps\\7_Advanced_ML_Pipeline\\End-to-End-Machine-Learning-Pipeline'

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path